## Gradio

In [97]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
!ls "/content/drive/My Drive/Demo"

 hotel_efficientnet.pkl      hotel_resnet34.pkl
 hotel_id_code_mapping.csv  'ResNet Gradio.ipynb'


In [99]:
# 1) Point at your Drive folder
DRIVE_FOLDER      = "/content/drive/My Drive/Demo"
WEIGHTS_PATH      = f"{DRIVE_FOLDER}/hotel_resnet34.pkl"
MAPPING_CSV_PATH  = f"{DRIVE_FOLDER}/hotel_id_code_mapping.csv"

In [100]:
# 2) Reload everything from Drive
import timm, pandas as pd, numpy as np, cv2
from albumentations import Compose, ToFloat
from albumentations.pytorch import ToTensorV2
from PIL import Image
import torch


In [101]:
# 2a) load your hotel_id mapping
mapping_df        = pd.read_csv(MAPPING_CSV_PATH)
hotel_id_code_map = mapping_df.set_index("hotel_id_code")["hotel_id"].to_dict()


In [102]:
# 2b) redefine your model class
class HotelIdModel(torch.nn.Module):
    def __init__(self, n_classes, backbone_name="resnet34"):
        super().__init__()
        self.backbone = timm.create_model(backbone_name,
                                          num_classes=n_classes,
                                          pretrained=False)
    def forward(self, x):
        return self.backbone(x)

In [103]:
# 2c) instantiate + load weights ResNet
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_classes = len(hotel_id_code_map)
model    = HotelIdModel(n_classes, "resnet34")
state    = torch.load(WEIGHTS_PATH, map_location=device)
model.load_state_dict(state)
model.to(device).eval()
print("✅ ResNet model reloaded and ready")

✅ ResNet model reloaded and ready


In [104]:
# 3) your preprocessing & TTA predict helper (unchanged)
base_transform = Compose([ ToFloat(), ToTensorV2() ])

def pad_and_resize(img, size=256):
    h, w = img.shape[:2]
    diff = abs(h - w)//2
    if h > w:
        img = cv2.copyMakeBorder(img, 0,0, diff,diff, cv2.BORDER_CONSTANT)
    else:
        img = cv2.copyMakeBorder(img, diff,diff,0,0, cv2.BORDER_CONSTANT)
    return cv2.resize(img, (size,size))

def predict_top5(pil_img):
    img    = np.array(pil_img)                # PIL→RGB numpy
    img    = pad_and_resize(img, size=256)
    tensor = base_transform(image=img)["image"].unsqueeze(0).to(device)
    with torch.no_grad():
        out1 = torch.sigmoid(model(tensor))
        out2 = torch.sigmoid(model(torch.flip(tensor, dims=[-1])))
        avg  = (out1 + out2)/2
    idxs = torch.topk(avg, k=5, dim=1).indices.cpu().numpy().ravel()
    return [hotel_id_code_map[int(i)] for i in idxs]

In [105]:
!pip install gradio

In [108]:
import gradio as gr

#css = ".gradio-container {background: white; color:black;}"

css = """
.gradio-container {background: white; color:black;}
h1 {color: white; background: #0097a7; font-weight: normal; font-family: Arial, Helvetica, sans-serif; font-size:40px;}
p {color: #0097a7; font-family: Arial, Helvetica, sans-serif; font-size:22px; font-weight: bold;}
"""

iface = gr.Interface(
    theme = gr.themes.Ocean(),
    css=css,
    fn=predict_top5,
    inputs=[
    #     gr.Dropdown(
    #     ["ResNet 34", "EfficientNet"],
    #     label="Model",
    #     multiselect=False,
    # ),
            gr.Image(type="pil", label="Upload Image")],
    outputs=gr.JSON(label="Top 5 Hotel IDs"),
    title="Hotel Identification",
    description="Upload a hotel room photo and get the top-5 predicted hotel IDs."
)
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b89c8a64d80eafa180.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
